Link to Github repository: https://github.com/fridamariejoergensen/CSS.git <br>
### Contribution Statement

We, Frida(s206182), Cecilie(sXXXXXX), and Marie(sXXXXXX), collaborated on Assignment 1 in Jupyter Notebook and made the following contributions:

- Cecilie focused on this

- Frida did this

- Marie did this



# Assignment 1 - "name"

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np
import PyPDF2
import pickle

## 2019 edition

In [ ]:
## Poster
LINK = "https://2019.ic2s2.org/posters/"
r = requests.get(LINK) 
soup = BeautifulSoup(r.content)

In [ ]:
listul = soup.find("div",{"class":"col-md-8 page-content-wrap"})
listli = listul.find_all("li")

allnames = []
for i in range(len(listli)):
    if listli[i].find("strong"):
        listli[i].strong.extract()
    if listli[i].find("span"):
        listli[i].span.extract()
    if listli[i].find("br"):
        listli[i].br.extract()
        
    names = listli[i].text.replace("\n","").replace(" and ",",").replace(", ",",")
    allnames = allnames + names.split(",")
    


un2019 = np.unique(np.array(allnames))
len(un2019)

In [ ]:
#### ORal
LINK = 'https://2019.ic2s2.org/oral-presentations/'
r = requests.get(LINK)
all_oral = BeautifulSoup(r.content)

In [ ]:
# extract the specific element of the site that contains the names
paragraphs = all_oral.find_all("p", {"class":""})[3:39]

In [ ]:
names = []
for p in paragraphs:
    text = p.text.strip()
    lines = text.split(' – ')
    names.extend([line.split('.')[0] for line in lines[1:]])

In [ ]:
# some code that extracts list elements that contain numbers.

def remove_elements_with_numbers(names):
    return [i for i in names if not any(j.isdigit() for j in i)]

p = remove_elements_with_numbers(names)

In [2]:
newlist = [word for line in p for word in line.split(',')]
newlist.remove('No Presentation')
final_list = np.asarray(newlist)
print(len(np.unique(final_list)))

NameError: name 'p' is not defined

In [ ]:
combined_list = np.concatenate((final_list, un2019))
print(len(np.unique(combined_list)))

## 2020 edition

In [ ]:
LINK = "https://docs.google.com/spreadsheets/u/0/d/e/2PACX-1vTX9_1Xftn7D-nSI8X9b7tafr_Z0kAbphKdfZ8qUSU9p-syXNsGPdhHl5ZyTnKKL-T6dCEJqtsrn3wy/pubhtml/sheet?headers=false&gid=181378784"
r = requests.get(LINK)
soup = BeautifulSoup(r.content)

In [ ]:
table = soup.find("table")
   
#HERE I GET THE ROWS
allnames = []
tabrows = table.tbody.find_all("tr")

for tr in tabrows[1:]:
    tds = tr.find_all('td')
    names = tds[2].text.replace(", ",",").replace(" and ",",")
    allnames = allnames + names.split(",")

un2020 = np.unique(np.array(allnames))
len(un2020)

## 2021 edition

In [ ]:
## All authors 2021
url = 'https://easychair.org/smart-program/IC2S2-2021/talk_author_index.html#let9'
r2 = requests.get(url)
soup2 = BeautifulSoup(r2.content)

In [ ]:
# Finding table holding the data
table = soup2.find("table",{"class":"index"})
table_rows = table.find_all("tr")

In [ ]:
# Subtracting all names into a list
rows = []
for tr in table_rows[1:]:
    tds = tr.find_all('td', {'class':'name'})
    row = [td.text.replace("\n","") for td in tds]
    rows.append(row)

In [ ]:
# Splitting strings in list into more strings
new_rows = []
for i in range(len(rows)):
    if len(rows[i]) > 0:
        new_rows.append(rows[i][0].split(','))
    else:
        continue

In [ ]:
# Reversing order of front name and last name in lists
for i in range(len(new_rows)):
    pos = new_rows[i][0]
    new_rows[i][0] = new_rows[i][1]
    new_rows[i][1] = pos

In [ ]:
# Removing nested lists and joining strings
result = []
for i in new_rows:
    rev = ' '.join(i)
    result.append(rev)

print(len(np.unique(result))) # = 692

In [ ]:
list_all_years = np.unique(np.concatenate((final_list, un2019,un2020,result)))
print(len(list_all_years))

In [ ]:
pickle.dump(list_all_years,open("pickles/conference_names","wb"))